# Phase 2: Find Processing Techniques
## Iteration 3: Find Best Preprocessing Techniques 
### Import Libraries and Scripts

In [6]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# import functions and classes for model
from Utils.preporcessing_utils import data_loading 
from Utils.preporcessing_utils import labels_encoding
from Utils.preporcessing_utils import split_data
from Utils.preporcessing_utils import ablation
from Utils.preporcessing_utils_Copy1 import combination
from Utils.run_model_utils import run_model

In [ ]:
# import functions and classes for visualization
from Utils.visualizations_utils import Visualization

### Data Preparation and Basic Preprocessing

In [ ]:
# Loads data
train_df, test_df = data_loading("train_full.csv", "test_full.csv")

In [ ]:
# Create dataframe and transform(encodes) pathology labels
train_df, test_df = labels_encoding(train_df, test_df)

In [ ]:
train_df.columns

In [ ]:
# Split data
train_data, val_data, test_data = split_data(train_df, test_df, 0.11)

### Create Preporcessing Groups By Using Combinations

In [ ]:
# create groups using ablation
options_c = ['apply_background_removal',
           'apply_crop',
           'apply_noise_reduction',
           'apply_contrast_enhancement',
           'apply_edge_enhancement',
           'apply_lbp_texturizer']

In [ ]:
# create group of techniques to try
techniques_groups = combination(options_c)

In [ ]:
techniques_groups["Baseline Basic Preporcessing"]

### Fine-Tune Custom CNN Settings 

In [ ]:
architecture_settings_iter3 = {
    "Baseline_3.2(DenseU=256, Dropout=0.4, layer=256)": {"epochs":12, "layers": [32, 64, 128, 256], "activation": 'relu', "dense_units":[256], "dropout": 0.4}
}

In [ ]:
import pandas as pd
arch = pd.DataFrame(architecture_settings_iter3)
arch.transpose()

### Trining Dynamic Custom Model with Preprocessing Techniques, 12 Epochs and Best Settings
#### Training
This models uses the custom CNN with the tf.Dataset image iterator, preporcessing techniques, 12 epochs and best settings. <br>
Settings:
- Dropout from 0.4
- Dense units of 256
- Adding a new layer of 256 

In [ ]:
# runs baseline model with an image dataset generator 
run_model((train_data, val_data, test_data), 
              techniques_groups, 
              project_phase="P2",
              iteration=3.2,
              with_preprocess=False, 
              model_type="dynamic",
              models_settings=architecture_settings_iter3)

#### Visulaizing
The following are some visualizations to compare the model outputs for each techniques

In [ ]:
# gets data based from models using tf.Data.Dataset
out_dir = "Outputs/Iteration3.2"

# initialize visualization instance
vis = Visualization(out_directory=out_dir, identifier=1)

# loads data 
models_data = vis.load_files()

# gets names of models
display_models_names = models_data.keys()

# classes
classes = ['Benigant', 'Malignant']

In [ ]:
# vusualize confusion matrix
vis.confusion_matrices(models_data, models_data.keys(), classes=classes)

In [ ]:
# visualize line plots 
vis.learning_curves(models_data, display_models_names, ["accuracy", "loss"], ["blue", "orange"])

In [ ]:
vis.line_plot(models_data, display_models_names, ["accuracy", "precision", "recall", "f1_score", "roc_auc", "specificity"])

In [ ]:
vis.line_plot(models_data, display_models_names, ["fpr", "fnr"])

In [ ]:
vis.radar_chart(models_data, display_models_names, ["accuracy", "precision", "recall", "f1_score", "roc_auc", "specificity"])

In [ ]:
vis.ROC_curve(models_data, display_models_names)